In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-zb4gcsw0 because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
from visuals import *
from utils import *
from model import *
from dataset import *
from data_module import *

In [3]:
plt.rcParams['figure.figsize'] = (14,10)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 22}
plt.rc('font', **font)

In [4]:
df = pd.read_csv('dataset_strat.csv')

In [5]:
df["bboxes"] = df["bboxes"].apply(eval) # DONT forget this cell

In [6]:
#dm = BoltDataModule(df, num_workers = 16, bs = 2)

In [7]:
model = BoltDetector(df, bs=3, num_classes=1, img_size=1024)

{'name': 'tf_efficientnetv2_l', 'backbone_name': 'tf_efficientnetv2_l', 'backbone_args': {'drop_path_rate': 0.2}, 'backbone_indices': None, 'image_size': [1024, 1024], 'num_classes': 1, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': 'same', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 3, 'fpn_cell_repeats': 3, 'fpn_channels': 88, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': True, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, 'box_loss_weight': 50.0, 'soft_nms': False, 'max_detection_points': 5000, 'max_det_per_image': 100, 'url': '

In [8]:
#path = '/media/scratch/astamoulakatos/NMIS/detector/detector-epoch=83-valid_loss=0.57.ckpt'
# Get the latest model
#path = '/media/scratch/astamoulakatos/NMIS/detector/detector-epoch=83-valid_loss=0.57.ckpt'
import os

for root, dirs, files in os.walk("./detector", topdown=False):
  for name in files:
    path= os.path.join(os.getcwd(), "detector", name)
    break

print(path)

/home/jovyan/work/COREF/src/Bolt_Detection/detector/epoch=8-valid_loss=1.46.ckpt


In [9]:
model = model.load_from_checkpoint(path, map_location='cuda:0')

/opt/conda/lib/python3.9/site-packages/torch/cuda/__init__.py:145: UserWarning: 
NVIDIA GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


{'name': 'tf_efficientnetv2_l', 'backbone_name': 'tf_efficientnetv2_l', 'backbone_args': {'drop_path_rate': 0.2}, 'backbone_indices': None, 'image_size': [1024, 1024], 'num_classes': 1, 'min_level': 3, 'max_level': 7, 'num_levels': 5, 'num_scales': 3, 'aspect_ratios': [[1.0, 1.0], [1.4, 0.7], [0.7, 1.4]], 'anchor_scale': 4.0, 'pad_type': 'same', 'act_type': 'swish', 'norm_layer': None, 'norm_kwargs': {'eps': 0.001, 'momentum': 0.01}, 'box_class_repeats': 3, 'fpn_cell_repeats': 3, 'fpn_channels': 88, 'separable_conv': True, 'apply_resample_bn': True, 'conv_bn_relu_pattern': False, 'downsample_type': 'max', 'upsample_type': 'nearest', 'redundant_bias': True, 'head_bn_level_first': False, 'head_act_type': None, 'fpn_name': None, 'fpn_config': None, 'fpn_drop_path_rate': 0.0, 'alpha': 0.25, 'gamma': 1.5, 'label_smoothing': 0.0, 'legacy_focal': False, 'jit_loss': False, 'delta': 0.1, 'box_loss_weight': 50.0, 'soft_nms': False, 'max_detection_points': 5000, 'max_det_per_image': 100, 'url': '

In [10]:
model.eval();

In [11]:
loader = model.val_dataloader()

In [12]:
outputs = []
for batch in loader:
    output = model.validation_step(batch=batch, batch_idx=0)
    outputs.append(output)

/opt/conda/lib/python3.9/site-packages/effdet/bench.py:55: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  indices_all = cls_topk_indices_all // num_classes
/opt/conda/lib/python3.9/site-packages/pytorch_lightning/core/lightning.py:374: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


In [16]:
len(outputs)

18

In [17]:
model.validation_epoch_end(outputs)

{'val_loss': tensor(1.4647),
 'metrics': {'AP_all': -1,
  'AP_all_IOU_0_50': -1,
  'AP_all_IOU_0_75': -1,
  'AP_small': -1,
  'AP_medium': -1,
  'AP_large': -1,
  'AR_all_dets_1': -1,
  'AR_all_dets_10': -1,
  'AR_all': -1,
  'AR_small': -1,
  'AR_medium': -1,
  'AR_large': -1}}

In [19]:
loader = model.test_dataloader()

In [20]:
outputs = []
for batch in loader:
    output = model.validation_step(batch=batch, batch_idx=0)
    outputs.append(output)

In [21]:
model.validation_epoch_end(outputs)

creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.39s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.347
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.559
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.422
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.347
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.003
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.038
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.390
 Average Recall     (AR) @[ IoU=0

{'val_loss': tensor(0.3298),
 'metrics': {'AP_all': 0.34736314309606564,
  'AP_all_IOU_0_50': 0.5592909613541999,
  'AP_all_IOU_0_75': 0.4217663324844231,
  'AP_small': 0.34736314309606564,
  'AP_medium': -1.0,
  'AP_large': -1.0,
  'AR_all_dets_1': 0.002890173410404624,
  'AR_all_dets_10': 0.03757225433526011,
  'AR_all': 0.3901734104046243,
  'AR_small': 0.3901734104046243,
  'AR_medium': -1.0,
  'AR_large': -1.0}}

In [22]:
model.device

device(type='cpu')

In [23]:
output['batch_predictions']

{'predictions': tensor([[[4.3359e+02, 3.1790e+02, 4.6926e+02, 3.4523e+02, 9.1606e-01,
           1.0000e+00],
          [1.9348e+02, 4.5812e+02, 2.2830e+02, 4.8432e+02, 8.8686e-01,
           1.0000e+00],
          [1.9573e+02, 3.7775e+01, 2.2913e+02, 6.3065e+01, 8.6961e-01,
           1.0000e+00],
          ...,
          [7.7896e+01, 1.1460e+02, 1.0592e+02, 1.5244e+02, 2.3341e-02,
           1.0000e+00],
          [5.3021e+01, 1.8847e+02, 7.9952e+01, 2.2465e+02, 2.2966e-02,
           1.0000e+00],
          [1.8094e+02, 2.4354e+02, 3.5121e+02, 2.8162e+02, 2.2759e-02,
           1.0000e+00]],
 
         [[4.3150e+02, 2.4583e+02, 4.5714e+02, 2.7936e+02, 6.1033e-01,
           1.0000e+00],
          [1.5861e+02, 7.9295e+00, 1.8407e+02, 4.0756e+01, 5.4415e-01,
           1.0000e+00],
          [3.9826e+01, 2.3946e+02, 6.6026e+01, 2.7401e+02, 4.8633e-01,
           1.0000e+00],
          ...,
          [1.9900e+02, 6.6271e+01, 2.2581e+02, 1.0245e+02, 4.2846e-02,
           1.0000e+00],
  